# Topics


In [1]:
import numpy as np
import pandas as pd
import math
import re
import sys
from shapely.geometry import Polygon
from matplotlib import pyplot as plt
from collections import Counter, OrderedDict, namedtuple, defaultdict, ChainMap
from queue import Queue
from copy import deepcopy
import networkx as nx
from functools import cmp_to_key, reduce
from itertools import product, permutations, combinations, combinations_with_replacement
from itertools import repeat
from functools import cache
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import maximum_flow
import json
import time
from tqdm import tqdm

In [2]:
sys.setrecursionlimit(1500)

In [3]:
with open("12-input", "r") as file:
    lines = file.readlines()
data_raw = "".join(lines)
# data_raw = [line.replace("\n", "") for line in lines]
# data_raw = "\n".join(data_raw)
data_raw

'SSSSSSSSSSSSSSSWWWWWHHHHHHHHHHHHHHHHHHBBBJJJJJJJJJIJUUUUUUUUUOOOOOOOOOHHHHHHHHHGGGGGGGGGGGGGGGGTTTTTTTTTTZZZHHSSHHHEEEEEEEETTTTTTTTTTOOOOOOO\nSSSSSSSSSSSSSSSWWWWWWHHHHHHHHHHHHHHHHHBBJJJJJJJJJJJJUUUUUUUOOOOOOOOOOOOHHHHHHHHHGGGGGGGGGGTTTGTTTTTTTTTTZZHHHSSSHHEEEEEEEEETTTTTTTTTTTTOOOOO\nSSSSSSSSSSSSAAWWWWWWWBBBBHBBBHHHHHHHHHJJJJJJJJJJJJJUUUUUUUPOCOOOOOOOOOOHHHHHHHHHGGGGGGGGGGGTTTTTTTTTTTTTZHHHHHHSHHHHEEEEEEETTTTTTTTTTTOOOOOO\nSSSSSSSSSSSSAAAAAWRWBBBBBBBBBHHHHHHHHHJJJJJJJJJJJJJJUUUPPPPPPSOOOOOOOOOHOOOOHHHGGGGGGGGGGGGTTTTTTTTTTTTTHHHHHHHHHHHHBBEEEEETTTTTTTTTTTTOOOOO\nSSSSSSSSSSAAAAAAAWRWWBBBBBBHHHHHHHHHHHJJJCJJJJJJJJJJUUUUPPPPPOOOOOOOOOOOOOOOHHHGGGGGGGGGGGGTTTTTTTTTTTTTHHHHHHHHHHHBBBBBEEETTTTTTTTOOOOOOOOO\nSSSSSSSSAAAAAAAARRRWBBBBBBBHHHHHHHHHHHJJJJJJJJJJJJJJUUUUPPPPPOOOOOOOOOOOOOOOHGHGGGGGGGGGGGGETTTTTTTTTTTTHHHHHHHHEHHBBBBBEEEEEOHTTTTOOOOOOOOO\nSSSSSSSAAAAAAAARRRRBBAABBBZZHHHHHHHHCCXXJJJJJJJJJJUUUUUUPPPPPPPOOOOOOOOOOOOOGGGBGGGGGGGGGGGETTTTTTTTTTTTHHGGGHHHHBBBBBBBEEEEEOOTOTOOOOOOOOOO\nSSSSS

In [20]:
test_data_raw = r"""AAAA
BBCD
BBCC
EEEC"""

# test_data_raw2 = r"""AAAAXXX
# XXXXXXD
# BBXXCXX
# BBXXCCX
# XXXXXCX
# EEEXXXX"""

test_data_raw2 = r"""OOOOO
OXOXO
OOOOO
OXOXO
OOOOO"""

test_data_raw3 = r"""RRRRIICCFF
RRRRIICCCF
VVRRRCCFFF
VVRCCCJFFF
VVVVCJJCFE
VVIVCCJJEE
VVIIICJJEE
MIIIIIJJEE
MIIISIJEEE
MMMISSJEEE"""

test_data_raw4 = r"""EEEEE
EXXXX
EEEEE
EXXXX
EEEEE"""

test_data_raw5 = r"""AAAAAA
AAABBA
AAABBA
ABBAAA
ABBAAA
AAAAAA"""

def preprocess_data (data):
    rows = [list(row) for row in data.split("\n")]
    return np.array(rows, dtype='U10')



test_data = preprocess_data(test_data_raw)
test_data2 = preprocess_data(test_data_raw2)
test_data3 = preprocess_data(test_data_raw3)
test_data4 = preprocess_data(test_data_raw4)
test_data5 = preprocess_data(test_data_raw5)
display(test_data)
display(test_data2)
display(test_data3)
display(test_data4)
display(test_data5)

array([['A', 'A', 'A', 'A'],
       ['B', 'B', 'C', 'D'],
       ['B', 'B', 'C', 'C'],
       ['E', 'E', 'E', 'C']], dtype='<U10')

array([['O', 'O', 'O', 'O', 'O'],
       ['O', 'X', 'O', 'X', 'O'],
       ['O', 'O', 'O', 'O', 'O'],
       ['O', 'X', 'O', 'X', 'O'],
       ['O', 'O', 'O', 'O', 'O']], dtype='<U10')

array([['R', 'R', 'R', 'R', 'I', 'I', 'C', 'C', 'F', 'F'],
       ['R', 'R', 'R', 'R', 'I', 'I', 'C', 'C', 'C', 'F'],
       ['V', 'V', 'R', 'R', 'R', 'C', 'C', 'F', 'F', 'F'],
       ['V', 'V', 'R', 'C', 'C', 'C', 'J', 'F', 'F', 'F'],
       ['V', 'V', 'V', 'V', 'C', 'J', 'J', 'C', 'F', 'E'],
       ['V', 'V', 'I', 'V', 'C', 'C', 'J', 'J', 'E', 'E'],
       ['V', 'V', 'I', 'I', 'I', 'C', 'J', 'J', 'E', 'E'],
       ['M', 'I', 'I', 'I', 'I', 'I', 'J', 'J', 'E', 'E'],
       ['M', 'I', 'I', 'I', 'S', 'I', 'J', 'E', 'E', 'E'],
       ['M', 'M', 'M', 'I', 'S', 'S', 'J', 'E', 'E', 'E']], dtype='<U10')

array([['E', 'E', 'E', 'E', 'E'],
       ['E', 'X', 'X', 'X', 'X'],
       ['E', 'E', 'E', 'E', 'E'],
       ['E', 'X', 'X', 'X', 'X'],
       ['E', 'E', 'E', 'E', 'E']], dtype='<U10')

array([['A', 'A', 'A', 'A', 'A', 'A'],
       ['A', 'A', 'A', 'B', 'B', 'A'],
       ['A', 'A', 'A', 'B', 'B', 'A'],
       ['A', 'B', 'B', 'A', 'A', 'A'],
       ['A', 'B', 'B', 'A', 'A', 'A'],
       ['A', 'A', 'A', 'A', 'A', 'A']], dtype='<U10')

In [6]:
full_data = preprocess_data(data_raw)
full_data

array([['S', 'S', 'S', ..., 'O', 'O', 'O'],
       ['S', 'S', 'S', ..., 'O', 'O', 'O'],
       ['S', 'S', 'S', ..., 'O', 'O', 'O'],
       ...,
       ['P', 'P', 'P', ..., 'O', 'O', 'O'],
       ['P', 'P', 'P', ..., 'O', 'O', 'O'],
       ['R', 'R', 'P', ..., 'O', 'O', 'O']], dtype='<U10')

In [16]:

def solution (data, verbose=False):
    
    def isValid(shape : tuple, index : tuple):
        return (0 <= index[0] < shape[0] and
                0 <= index[1] < shape[1])
    
    def getNeighbors (node, array):
        directions = [(1,0), (-1,0), (0,1), (0,-1)]
        next_nodes = [(node[0]+direction[0], node[1]+direction[1])  for direction in directions 
                                                                if isValid(array.shape, (node[0]+direction[0], node[1]+direction[1])) 
                                                                    and array[node] == array[(node[0]+direction[0], node[1]+direction[1])] ]
        return next_nodes
    
    def getFalseNeighbors (node, array):
        directions = [(1,0), (-1,0), (0,1), (0,-1)]
        next_nodes = [(node[0]+direction[0], node[1]+direction[1])  for direction in directions 
                                                                if isValid(array.shape, (node[0]+direction[0], node[1]+direction[1])) 
                                                                    and array[node] != array[(node[0]+direction[0], node[1]+direction[1])] ]
        return next_nodes
    
    
    total = 0
    
    visited = set()
    regions = []
    plant_types = np.unique(data)
    for plant in plant_types:
        positions = [tuple(pos)  for pos in np.argwhere(data == plant).tolist()]
        for start_position in positions:
            if start_position in visited:
                continue
            
            buffer_nodes = [start_position]
            region = []
            while buffer_nodes:
                current_note = buffer_nodes.pop()
                if current_note in visited:
                    continue
                
                visited.add(current_note)
                region.append(current_note)
                next_nodes = getNeighbors(current_note, data)
                buffer_nodes += next_nodes

            regions.append((data[start_position], region))
    
    # print(data)
    # print(regions)

    # calc perimeters
    for region in regions:
        num_fences = 0
        for plant in region[1]:
            other_neighbors = getNeighbors(plant, data)
            num_fences += 4 - len(other_neighbors)

        total += num_fences * len(region[1])
    
    return total



sol = solution(test_data, verbose=True)
print(sol)
sol = solution(test_data2, verbose=True)
print(sol)
sol = solution(test_data3, verbose=True)
print(sol)


140
1232
1930


In [17]:
sol = solution(full_data)
print(sol)

1473408


# Part 2

In [60]:
a = np.array([["A", "A", "B"], ["A", "B", "B"], ["A", "B", "A"]])
print(a)
idx = [(0,0),(0,1), (1,0)]
np.put(a, idx, '.')


[['A' 'A' 'B']
 ['A' 'B' 'B']
 ['A' 'B' 'A']]


In [66]:
b = np.full(a.shape, '.')
b

array([['.', '.', '.'],
       ['.', '.', '.'],
       ['.', '.', '.']], dtype='<U1')

In [71]:
def solution2 (data, verbose=False):

    def isValid(shape : tuple, index : tuple):
        return (0 <= index[0] < shape[0] and
                0 <= index[1] < shape[1])
    
    def getNeighbors (node, array):
        directions = [(1,0), (-1,0), (0,1), (0,-1)]
        next_nodes = [(node[0]+direction[0], node[1]+direction[1])  for direction in directions 
                                                                if isValid(array.shape, (node[0]+direction[0], node[1]+direction[1])) 
                                                                    and array[node] == array[(node[0]+direction[0], node[1]+direction[1])] ]
        return next_nodes
    
    def getFalseNeighbors (node, array):
        directions = [(1,0), (-1,0), (0,1), (0,-1)]
        next_nodes = [(node[0]+direction[0], node[1]+direction[1])  for direction in directions 
                                                                if isValid(array.shape, (node[0]+direction[0], node[1]+direction[1])) 
                                                                    and array[node] != array[(node[0]+direction[0], node[1]+direction[1])] ]
        return next_nodes
    
    
    total = 0
    
    visited = set()
    regions = []
    plant_types = np.unique(data)
    for plant in plant_types:
        positions = [tuple(pos)  for pos in np.argwhere(data == plant).tolist()]
        for start_position in positions:
            if start_position in visited:
                continue
            
            buffer_nodes = [start_position]
            region = []
            while buffer_nodes:
                current_note = buffer_nodes.pop()
                if current_note in visited:
                    continue
                
                visited.add(current_note)
                region.append(current_note)
                next_nodes = getNeighbors(current_note, data)
                buffer_nodes += next_nodes

            regions.append((data[start_position], region))
    

    def isValidPlant(array : np.ndarray, index : tuple, plant: str):
        return (0 <= index[0] < array.shape[0] and
                0 <= index[1] < array.shape[1] and 
                array[index] == plant) 
    
    # calc perimeters
    for region in regions:
        num_fences = 0
        
        region_data = np.full(data.shape, ".")
        for pos in region[1]:
            region_data[pos] = region[0]
        
        x_coords = [pos[0] for pos in region[1]]
        y_coords = [pos[1] for pos in region[1]]
        x_min, x_max = min(x_coords), max(x_coords)
        y_min, y_max = min(y_coords), max(y_coords)
        
        
        # Horizontal fences - only count top fence 
        for x in range(x_min, x_max+2):
            fencing = False
            top_plant = False
            layer_fences = 0
            
            for y in range(y_min, y_max+1):
                top = isValidPlant(region_data, (x-1, y), region[0])
                bot = isValidPlant(region_data, (x, y), region[0])
                # print(top, bot)
                if (top and bot) or (not top and not bot):
                    if fencing:
                        fencing = False
                        layer_fences += 1
                        # print("Fence Ended")
                elif (top and not bot):
                    if not fencing:
                        fencing = True
                        top_plant = True
                    else:
                        # Special case of diagonal connection
                        if not top_plant:
                            top_plant = True
                            layer_fences += 1
                            # print("Fence Ended diagonally")
                elif (not top and bot):
                    if not fencing:
                        fencing = True
                        top_plant = False
                    else:
                        # Special case of diagonal connection
                        if top_plant:
                            top_plant = False
                            layer_fences += 1
                            # print("Fence Ended diagonally")
            if fencing:
                layer_fences += 1
                # print("Fence Ended at region end")

            num_fences += layer_fences
        
        num_fences_save = num_fences
        
        # Vertical fences
        for y in range(y_min, y_max+2):
            fencing = False
            left_plant = False
            layer_fences = 0
            
            for x in range(x_min, x_max+1):
                left = isValidPlant(region_data, (x, y-1), region[0])
                right = isValidPlant(region_data, (x, y), region[0])
                # print(right, left)
                if (left and right) or (not left and not right):
                    if fencing:
                        fencing = False
                        layer_fences += 1
                        # print("*** Fence Ended")
                elif (left and not right):
                    if not fencing:
                        fencing = True
                        left_plant = True
                    else:
                        # Special case of diagonal connection
                        if not left_plant:
                            left_plant = True
                            layer_fences += 1
                            # print("*** Fence Ended diagonally")
                elif (not left and right):
                    if not fencing:
                        fencing = True
                        left_plant = False
                    else:
                        # Special case of diagonal connection
                        if left_plant:
                            left_plant = False
                            layer_fences += 1
                            # print("*** Fence Ended diagonally")
            if fencing:
                layer_fences += 1
                # print("*** Fence Ended at region end")
            num_fences += layer_fences
        
        
        total += num_fences * len(region[1])
    
    return total




sol = solution2(test_data, verbose=True)
print("Solution 80", sol)
sol = solution2(test_data2, verbose=True)
print("Solution 436",sol)
sol = solution2(test_data3, verbose=True)
print("Solution 1206",sol)
sol = solution2(test_data4, verbose=True)
print("Solution 236",sol)
sol = solution2(test_data5, verbose=True)
print("Solution 368",sol)

    

Solution 80 80
Solution 436 436
Solution 1206 1206
Solution 236 236
Solution 368 368


In [72]:
sol = solution2(full_data)
print(sol)

886364
